# **Permutation Test for Metaheuristics** 
*   Omar Tawfik
*   Ahmed Aldabagh

The following notebook will demonstrate the statistical tests and algorithm written that computes the p-value using permutation tests. 

## Data Import


In [ ]:
#If you are using Google colab, the data samples can be zipped into one folder and uploaded. Followed by running the code:
from zipfile import ZipFile
file_name = './Sample.zip' #Change file name 

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

## Package Imports

In [ ]:
# importing the necessary libraries
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests
from itertools import combinations 

## Plots Settings



In [ ]:
# modifying the settings of all plots for the best visualization of data
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

## Options

Firstly, choose the function you want to perform the statistical tests on, choose the number of permutations, then choose the pairs you want to test on

In [ ]:
func_num = "1" #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
num_of_permutations = 1000000 #@param {type:"slider", min:10000, max:10000000, step:1}
#@markdown Write pairs as "Shade, L-Shade / JS, HBA" for example (without quotation marks)
pairs = "Shade, L-Shade / Shade, Jaya / JS, HBA" #@param {type:"string"}

pairs = [[y.strip() for y in x.strip().split(',')] for x in pairs.split('/')]

## Data Files Imports

In [ ]:
# importing the specific function files using the function number taken from above
data = {"Shade":[float(x) for x in (open("./Shade/SHADE_function_" + func_num + ".txt").readlines()[1:])], 
        "JS":[float(x) for x in (open("./JS/JS_function_" + func_num + ".txt").readlines()[1:])], 
        "Jaya":[float(x) for x in (open("./Jaya/JAYA_function_" + func_num + ".txt").readlines()[1:])],
        "HBA":[float(x) for x in (open("./HBA/HBA_function_" + func_num + ".txt").readlines()[1:])],
        "L-Shade":[float(x) for x in (open("./L-Shade/LSHADE_function_" + func_num + ".txt").readlines()[1:])]}

## Function Visualization

In [ ]:
# displaying a line graph with all methods over the 50 runs to observe patterns
plt.plot(data["Jaya"], label = "Jaya")
plt.plot(data["JS"], label = "JS")
plt.plot(data["Shade"], label = "Shade")
plt.plot(data["L-Shade"], label = "L-Shade")
plt.plot(data["HBA"], label = "HBA")



# adding a title and labels for clarification
plt.title("Function " + func_num)
plt.xlabel("Num of Runs")
plt.ylabel("Returned Result")

plt.yscale("log") # the values of some functions are too large hence the log is used
plt.legend()
plt.show()

In [ ]:
# displaying a scatter plot to know exactly what each method returned over the 50 runs
plt.scatter([x for x in range(1, 51)], data["Jaya"], facecolors="none", ec="#1f77b4", marker="o", label = "Jaya")
plt.scatter([x for x in range(1, 51)], data["JS"], facecolors="none", ec="orange", marker="x", label = "JS")
plt.scatter([x for x in range(1, 51)], data["Shade"], facecolors="none", ec="green", marker="s", label = "Shade")
plt.scatter([x for x in range(1, 51)], data["L-Shade"], facecolors="none", ec="red", marker="x", label = "L-Shade")
plt.scatter([x for x in range(1, 51)], data["HBA"], facecolors="none", ec="black", marker="s", label = "HBA")



plt.title("Function " + func_num)
plt.xlabel("Num of Runs")
plt.ylabel("Returned Result (log)")

plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
# displaying a boxplot for all methods
fig, ax = plt.subplots()
ax.boxplot(data.values())
ax.set_xticklabels(data.keys())

plt.title("Function " + func_num)
plt.xlabel("Optimization Methods")
plt.ylabel("Returned Result (log)")

plt.yscale("log")
plt.show()

## Descriptive Statistics

In [ ]:
# displaying a dataframe (table) comparing the mean, median, and the standard deviation of each method
d = [["{:.2E}".format(np.mean(data["Shade"])), "{:.2E}".format(np.median(data["Shade"])), "{:.2E}".format(np.std(data["Shade"])), "{:.2E}".format(np.max(data["Shade"])), "{:.2E}".format(np.min(data["Shade"]))],
    ["{:.2E}".format(np.mean(data["Jaya"])), "{:.2E}".format(np.median(data["Jaya"])), "{:.2E}".format(np.std(data["Jaya"])), "{:.2E}".format(np.max(data["Jaya"])), "{:.2E}".format(np.min(data["Jaya"]))],   
    ["{:.2E}".format(np.mean(data["JS"])), "{:.2E}".format(np.median(data["JS"])), "{:.2E}".format(np.std(data["JS"])), "{:.2E}".format(np.max(data["JS"])), "{:.2E}".format(np.min(data["JS"]))],
    ["{:.2E}".format(np.mean(data["L-Shade"])), "{:.2E}".format(np.median(data["L-Shade"])), "{:.2E}".format(np.std(data["L-Shade"])), "{:.2E}".format(np.max(data["L-Shade"])), "{:.2E}".format(np.min(data["L-Shade"]))],
    ["{:.2E}".format(np.mean(data["HBA"])), "{:.2E}".format(np.median(data["HBA"])), "{:.2E}".format(np.std(data["HBA"])), "{:.2E}".format(np.max(data["HBA"])), "{:.2E}".format(np.min(data["HBA"]))]]


display(pd.DataFrame(d, columns=["Mean", "Median", "Standard Deviation", "Max", "Min"], index=["Shade", "Jaya", "JS", "L-Shade", "HBA"]).sort_values("Mean"))

## Code

In [ ]:
# preparing the dataframes (tables)
df1 = pd.DataFrame(index=[[', '.join(x) for x in pairs]], columns=["KS-Test P-Value", "Sum-Rank P-Value", "Permutations P-Value"])
df2 = pd.DataFrame(index=[[', '.join(x) for x in pairs]], columns=["KS-Test P-Value", "Sum-Rank P-Value", "Permutations P-Value"])
df3 = pd.DataFrame(index=[[', '.join(x) for x in pairs]], columns=["KS-Test P-Value", "Sum-Rank P-Value", "Permutations P-Value"])

The following code will perform:
  - The permutation test for each of the chosen pairs
  - P-Value comparison between the KS-Test, Sum-Rank Test, and the Permutation Test
  - Bonferroni Correction Test
  - Benjamini-Hochberg Correction Test
  - Show permutation distributions for the pairs

In [ ]:
np.random.seed(123456789)
# looping through all possible pairs
for l1, l2 in combinations(data.keys(), 2):

  found = 0
  for x in pairs:
    if ((x[0] == l1 or x[0] == l2) and (x[1] == l1 or x[1] == l2)):
      found = 1

  if (found == 0):
    continue

  diff = np.abs(np.median (data[l1]) - np.median (data[l2])) #calculating the absolute difference of meadians of the pair

  diffs = [] # new empty list to store new differences in

  # loop for number of permutations chosen above
  for x in range(num_of_permutations):
    comb = np.random.permutation(np.concatenate((data[l1], data[l2]))) # combining the two lists and performing permutations

    # splitting the list to 2 new lists
    a = comb[:50]
    b = comb[50:]

    # calculating the medians of the new lists
    m1 = np.median(a)
    m2 = np.median(b)

    diffs.append (np.abs(m1 - m2)) # adding the new absolute difference to the differences list

  sum = len([x for x in diffs if x >= diff]) # calculating the sum of new differences greater than the original one

  p = sum / (num_of_permutations + 1) # calculating the p value of permutations
  ks, ks_p = sp.ks_2samp(data[l1], data[l2]) # performing a ks-test
  sum, sum_p = sp.ranksums(data[l1], data[l2]) # performing a t-test

  corrected_p = multipletests([p, ks_p, sum_p], alpha=0.05, method="bonferroni", is_sorted=False) # performing a Bonferroni correction
  corrected_p2 = multipletests([p, ks_p, sum_p], alpha=0.05, method="fdr_bh", is_sorted=False) # performing a Benjamini-Hochberg correction

  # Adding values to the dataframes (tables)
  df1.loc[l1 + ", " + l2,:] = ["{:.2E}".format(ks_p), "{:.2E}".format(sum_p), "{:.2E}".format(p)]
  df2.loc[l1 + ", " + l2,:] = ["{:.2E}".format(corrected_p[1][1]), "{:.2E}".format(corrected_p[1][2]), "{:.2E}".format(corrected_p[1][0])]
  df3.loc[l1 + ", " + l2,:] = ["{:.2E}".format(corrected_p2[1][1]), "{:.2E}".format(corrected_p2[1][2]), "{:.2E}".format(corrected_p2[1][0])]

  # plotting the permutations distribution
  plt.hist(diffs, facecolor = 'None', edgecolor='red', bins=25)
  plt.axvline(diff, color='blue')

  plt.title("Permutations Distribution for Function " + str(func_num) + "\n" + l1 + " vs. " + l2)
  plt.xlabel("Permutations Differences")
  plt.ylabel("Frequency")
  
  plt.show()
  plt.close()

  print()

In [ ]:
# displaying the dataframes (tables)
print("- Original Values:\n")
display(df1)

print("\n\n\n- Bonferroni Correction:\n")
display(df2)

print("\n\n\n- Benjamini-Hochberg Correction:\n")
display(df3)